In [6]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [14]:
import pickle
import pandas as pd
import numpy as np

In [8]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [9]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [40]:
year  = 2022
month = 2
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [41]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [42]:
# ------------------------------------------------------------ #
# Q1: What's the standard deviation of the predicted duration? #
# ------------------------------------------------------------ #
np.std(y_pred)

5.28140357655334

In [43]:
# Let's create an artificial 'ride_id' column:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [44]:
# Next, write the 'ride_id' and the predictions to a dataframe with results.
# (Save it as parquet):

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

output_file = f'./yellow_{year:04d}-{month:02d}.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [30]:
df_result

,ride_id,prediction
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904
...,...,...
2979426,2022/02_2979426,12.038225
2979427,2022/02_2979427,11.441569
2979428,2022/02_2979428,11.890459
2979429,2022/02_2979429,15.102681


In [31]:
# ------------------------------------------------------------ #
# Q2: What's the size of the output file?                      #
# ------------------------------------------------------------ #
! ls -l --block-size=MB

total 61MB
-rw-rw-r-- 1 javi javi  1MB jun 12 18:46 Dockerfile
-rw-rw-r-- 1 javi javi  1MB jun 12 18:46 model.bin
-rw-rw-r-- 1 javi javi  1MB jun 27 16:04 starter.ipynb
-rw-rw-r-- 1 javi javi  1MB jun 27 15:35 Untitled.ipynb
-rw-rw-r-- 1 javi javi 60MB jun 27 16:03 yellow_2022-02


Now let's turn the notebook into a script.

In [32]:
# ------------------------------------------------------------ #
# Q3: Which command you need to execute for that?              #
# ------------------------------------------------------------ #

! jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 2265 bytes to starter.py


Now let's put everything into a virtual environment. We'll use ```pipenv``` for that. Install all the required libraries. Pay attention to the Scikit-Learn version: it should be ```scikit-learn==1.2.2```.

In [39]:
! pipenv install scikit-learn==1.2.2 numpy pandas --python=3.10.9

Creating a Pipfile for this project...
Installing scikit-learn==1.2.2...
Resolving scikit-learn==1.2.2...
Adding scikit-learn to Pipfile's [packages] ...
✔ Installation Succeeded-learn...
⠋ Installing scikit-learn...
Installing numpy...
Resolving numpy...
Adding numpy to Pipfile's [packages] ...
✔ Installation Succeeded..
⠋ Installing numpy...
Installing pandas...
Resolving pandas...
Adding pandas to Pipfile's [packages] ...
✔ Installation Succeeded...
⠋ Installing pandas...
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠙ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (aa53783ca11b3f3e37662ec3e5c57b31709461fd122f87521ef1e4c5c95fb255)!
Installing dependencies from Pipfile.lock (5fb255)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [ ]:
# ------------------------------------------------------------ #
# Q4: What's the first hash for the Scikit-Learn dependency    #
#     in Pipfile.lock?
# ------------------------------------------------------------ #

first_hash = 'sha256:065e9673e24e0dc5113e2dd2b4ca30c9d8aa2fa90f4c0597241c93b63130d233'

Let's now make the script configurable via CLI. We'll create two parameters: year and month.

Run the script for March 2022 (see ```starter.py```).

In [ ]:
# ------------------------------------------------------------ #
# Q5: What's the mean predicted duration?                      #
# ------------------------------------------------------------ #

mean_pred_march = 12.758556818790902

Finally, we'll package the script in the docker container. For that, you'll need to use a base image that we prepared. 

We pushed it to ```svizor/zoomcamp-model:mlops-3.10.0-slim```, which you should use as your base image. 

After adjusting the ```Dockerfile```:

```dockerfile
FROM svizor/zoomcamp-model:mlops-3.10.0-slim

RUN pip install -U pip

RUN pip install pipenv
RUN pip install pyarrow  # Needed to save .parquet files

WORKDIR /app

COPY [ "Pipfile", "Pipfile.lock", "./" ]
    
RUN pipenv install --system --deploy

COPY [ "starter.py", "./" ]

ENTRYPOINT [ "python", "starter.py", "2022", "4" ]
```

We then build the Docker Image with:

```$ docker build -t ride-duration-prediction-homework-04 .```

And run the container with:

```$ docker run -it --rm ride-duration-prediction-homework-04```

In [ ]:
# ------------------------------------------------------------ #
# Q6: What's the mean predicted duration for April 2022?       #
# ------------------------------------------------------------ #

mean_pred_april = 12.827242870079969